# FEM Modelling for E-shaped Ferrite Core Transformer

## Import Packages

In [1]:
using gmsh
using LinearAlgebra
using LaTeXStrings
include("../../../../lib/Makie_Plots.jl");
using .Makie_Plots
include("../../../../lib/FEM_Tri_1e.jl");
using .FEM_Tri_1e
include("../../../../lib/Mesh_Data_e_core.jl");
using .Mesh_Data_e_core
include("../../../../lib/Save_VTK.jl");
using .Save_VTK

## Julia code

In [2]:
gmsh.initialize()

## Constants
mur = 2000;  #relative permeability of the core
mu0 = 4*pi*10^(-7);
curr_dens_value = 3.1085;
k = 0.2227

## Read mesh from file
gmsh.open("../mesh/e_shaped_ferrite_core_transformer.msh")

mesh_data = get_mesh_data_tri_1e(gmsh)

omega = 2*pi*50;  # Frequency

# Source current density J
sourcefunction(group_id) = curr_dens_value*(group_id==4)-curr_dens_value*(group_id==5)
sourceperelement = map(sourcefunction, mesh_data.e_group)

# Relative permeability model
reluctivityfunction(group_id) = (1/mu0)*((group_id==1)+(group_id==4)+(group_id==5))+(1/(mu0*mur))*((group_id==2))
reluctivityperelement = map(reluctivityfunction, mesh_data.e_group)

# Conductivity
conductivityfunction(group_id) = 0;
conductivityperelement = map(conductivityfunction, mesh_data.e_group);

# Handle the boundary conditions
node_ids1, node_coord, _ = gmsh.model.mesh.getNodes(0,1)
node_ids2, node_coord, _ = gmsh.model.mesh.getNodes(0,2)
node_ids3, node_coord, _ = gmsh.model.mesh.getNodes(0,3)
node_ids4, node_coord, _ = gmsh.model.mesh.getNodes(0,4)
node_ids5, node_coord, _ = gmsh.model.mesh.getNodes(1,101)
node_ids6, node_coord, _ = gmsh.model.mesh.getNodes(1,102)
node_ids7, node_coord, _ = gmsh.model.mesh.getNodes(1,103)
node_ids8, node_coord, _ = gmsh.model.mesh.getNodes(1,104)
bnd_node_ids = union(node_ids1,node_ids2,node_ids3,node_ids4,node_ids5,node_ids6,node_ids7,node_ids8)

# Calculate the vector potential
u = fem(mesh_data, sourceperelement, reluctivityperelement, conductivityperelement, omega, bnd_node_ids);

# Post-process for magnetic field and current density
Bx, By, B, Hx, Hy, H, mag_energy = post_process(mesh_data, u, reluctivityperelement);

# Contour plot of the magnetic flux density
plot_surface_tri_1e(norm.(B), "e_core_transformer_B_plot", gmsh)
# Contour plot of the magnetic field strength
plot_surface_tri_1e(norm.(H), "e_core_transformer_H_plot", gmsh)
# Contour plot of the magnetic energy
plot_surface_tri_1e(norm.(mag_energy), "e_core_transformer_mag_energy_plot", gmsh)

# Save as VTK file for Paraview visualization
save_vtk(mesh_data, norm.(u), norm.(B), norm.(H), norm.(mag_energy), reluctivityperelement, "e_core_transformer")

gmsh.finalize()

Info    : Reading '../mesh/e_shaped_ferrite_core_transformer.msh'...
Info    : 75 entities
Info    : 6027 nodes
Info    : 11928 elements
Info    : Done reading '../mesh/e_shaped_ferrite_core_transformer.msh'


### STEDIN Transformer : Magnetic field density
<img src="../img/e_core_transformer_B_plot.png">

### STEDIN Transformer : Magnetic field strength
<img src="../img/e_core_transformer_H_plot.png">

### STEDIN Transformer : Magnetic energy
<img src="../img/e_core_transformer_mag_energy_plot.png">